In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from nltk import Tree
from torch.distributions import Categorical
from torch.utils.tensorboard import SummaryWriter
from torch.optim.optimizer import required, Optimizer
from torch.autograd import grad
from torch.nn.utils import parameters_to_vector, vector_to_parameters
import torch
import os

In [2]:
class Encoder:
    """
    Description
    --------------
    Class represeting the one-hot encoder of the states.
    """
    
    def __init__(self, categories=[4, 3, 3, 3, 2, 4]):
        """
        Description
        --------------
        Constructor of class Encoder.
        
        Parameters & Attributes
        --------------
        categories : List of length d where categories[i] is the number of categories feature i can take.
        d          : Int, the input dimension.
        dim        : Int, the one-hot encoded representation dimension
        
        """
        
        self.categories = categories
        self.d = len(categories)
        self.dim = np.sum(categories) + self.d
        
    def transform(self, state_values):
        """
        Description
        --------------
        Encode the vector state representation with dummies.
        
        Parameters & Attributes
        --------------
        state_values : List of length d where the ith entry is either NaN or the the feature value.
        
        Returns
        --------------
        state_one_hot : 2D np.array representing the one-hot encoded state.
        """
    
        s = 0
        state_one_hot = np.zeros((1, self.dim), dtype = np.float32)
        for i, value in enumerate(state_values):
            if np.isnan(value):
                state_one_hot[0, self.categories[i] + s] = 1

            else:
                state_one_hot[0, value + s] = 1

            s += self.categories[i]+1

        return state_one_hot

In [3]:
class StateDQN:
    """
    Description
    --------------
    Class representing a state, it also serves as Node representation
    for our Breadth First Search
    """
    
    def __init__(self, values, encoder, categories=[4, 3, 3, 3, 2, 4]):
        """
        Description
        --------------
        Constructor of class State.
        
        Parameters & Attributes
        --------------
        values     : List of length d (Input dimension):
                         - values[i] = NaN if i is an unobsorved feature.
                         - values[i] = value of feature i if it is observed.
        encocer    : sklearn.preprocessing._encoders.OneHotEncoder object.
        categories : List of length d where categories[i] is the number of categories feature i can take.
        observed   : List containing the observed features at this state.
        unobserved : List containing the unobserved features at this state.
        empty      : Boolean, whether it is the empty state or not.
        complete   : Boolean, whether all features are observed or not.
        """
        
        d = len(values)
        values_nans = np.isnan(values)
        self.encoder = encoder  # One-hot encoder, used in the approximate RL framework for state representation.
        self.values = values
        self.values_encoded = self.encode()
        self.categories = categories
        self.observed = np.arange(d)[np.invert(values_nans)]
        self.unobserved = np.arange(d)[values_nans]  # These are also the allowed query actions at this state
        self.empty = (len(self.observed) == 0)
        self.complete = (len(self.unobserved) == 0)
        
    def encode(self):
        """
        Description
        --------------
        Encode the state with dummy variables. To be used when a one-hot encoder is defined.
        
        Parameters
        --------------
        
        Returns
        --------------
        np.array of shape (1, #one_hot_representation_dim)
        """
        
        return self.encoder.transform(self.values)
        
    def __repr__(self):
        """
        Description
        --------------
        String representation of the state.
        """
        
        s = '| '
        for x in self.values:
            s += str(x) + ' | '
            
        return s
    
    def __str__(self):
        """
        Description
        --------------
        String representation of the state.
        """
        
        s = '| '
        for x in self.values:
            s += str(x) + ' | '
            
        return s

In [4]:
class ActorCritic(nn.Module):
    
    def __init__(self, input_size=25, out=8):
        """
        Description
        ---------------
        Constructor of ActorCritic class.
        
        Parameters
        ---------------
        input_size    : Int, the one-hot encoding representation dimension.
        out           : Int, Actor output dimension, equal to the number of possible actions.
        fc_1          : nn.Linear, first fully connected layer (common parameters between Actor and Critic).
        fc_2          : nn.Linear, second fully connected layer (common parameters between Actor and Critic).
        actor_output  : nn.Linear, actor output fully connected layer.
        critic_output : nn.Linear, critic output fully connected layer.
        """
        
        super(ActorCritic, self).__init__()
        
        self.fc_1 = nn.Linear(input_size, 32)
        self.fc_2 = nn.Linear(32, 32)
        
        self.actor_output = nn.Linear(32, out)
        self.critic_output = nn.Linear(32, 1)
        
        nn.init.constant_(self.actor_output.weight, 0)
        nn.init.constant_(self.actor_output.bias, 0)
                        
    def forward(self, x):
        """
        Description
        ---------------
        The forward pass.
        
        Parameters
        ---------------
        x : torch.tensor of dimension (batch_size, input_size)
        
        Returns
        ---------------
        value_output  : torch.tensor of dimension (batch_size, 1)
        policy_output : torch.tensor of dimension (batch_size, out)
        """
        
        x = F.relu(self.fc_1(x))
        x = F.relu(self.fc_2(x))
        
        return self.critic_output(x), self.actor_output(x)

In [5]:
class EnvironmentDQN:
    """
    Description
    --------------
    Class representing the environment, it can generate data points that start each episode,
    keep track of the current state, return the reward of an action taken at the current state,
    and transition to the next corresponding state.
    """
    
    def __init__(self, generator, rewards_queries, encoder, r_plus=5, r_minus=-5, split=3):
        """
        Description
        --------------
        Constructor of class Environment.
        
        Parameters & Attributes
        --------------
        generator       : Dict, - keys   : Feature variables.
                                - values : List of probability masses of each category of the corresponding feature.
        rewards_queries : Dict, - keys   : Feature variables.
                                - values : Reward of querying the value of the corresponding feature.
        encoder         : Object of class Encoder, the encoder mapping states to their one-hot representation.
        r_plus          : Int, reward of a correct report (default=5).
        r_minus         : Int, reward of an incorrect report (default=-5).
        split           : Int, the split point we use to define our concept.
        d               : Int, the number of feature variables.
        data_point      : List of length d, the data point starting the episode.
        label           : Boolean, the true label of data_point.
        state           : Object of class State, the current state.
        done            : Boolean, whether the episode is finished or not.
        """
        
        self.generator = generator
        self.categories = [len(v) for v in self.generator.values()]
        self.d = len(self.categories)
        self.rewards_queries = rewards_queries
        self.encoder = encoder
        self.r_plus = r_plus
        self.r_minus = r_minus
        self.split = split
        
    def generate(self):
        """
        Description
        --------------
        Generate a data point.
        
        Parameters
        --------------
        
        Returns
        --------------
        List with the values of each feature, it represents the data point.
        """
        
        return [np.random.choice(self.categories[i], p=self.generator[i]) for i in range(self.d)]
    
    def concept(self, data_point):
        """
        Description
        --------------
        Define the concept labeling the data points. we can define it as a decision tree for example.
        
        Parameters
        --------------
        data_point : List of length d, the data point to label.
        
        Returns
        --------------
        Int in {0, 1}, the label of the data point.
        """
        
        label = True
        i = 0
        while label and i <= self.d-1:
            if data_point[i] >= self.split:
                label = False
                
            i += 1
            
        return label
    
    def reset(self, data_point=None):
        """
        Description
        --------------
        Reset the environment to start a new episode. If data_point is specified, start the episode from it,
        otherwise generate it.
        
        Parameters
        --------------
        data_point : List of length d, the data point to label (default=None).
        
        Returns
        --------------
        """
        
        self.data_point = self.generate() if data_point is None else data_point
        self.label = self.concept(self.data_point)
        self.state = StateDQN([np.NaN for i in range(self.d)], self.encoder, categories=self.categories)
        self.done = False
    
    def step(self, action):
        """
        Description
        --------------
        Interract with the environment through an action taken at the current state.
        
        Parameters
        --------------
        action : Int in {0, ..., d-1, d, d+1}, 
                 - 0, ..., d-1 represent query actions.
                 - d, d+1 represent report actions.
        
        Returns
        --------------
        reward     : Int, the reward of taking this action at the current state.
        next_state : Object of class State, the next state.
        done       : Boolean, whether the episode is finished or not.
        """
        
        # Treating query actions.
        if action <= self.d-1:
            # If it is an allowed query action.
            if np.isnan(self.state.values[action]):
                reward = self.rewards_queries[action]
                values = self.state.values
                values[action] = self.data_point[action] # Reveal the value of the queried feature in the data point.
                self.state = StateDQN(values, self.encoder, self.categories)
                
            # If this query action is not allowed.
            else:
                print('unallowed')
            
        # Treating report actions.
        else:
            reward = self.r_plus if (action%self.d) == self.label else self.r_minus
            self.done = True
            
        return reward, self.state, self.done

In [6]:
class AgentPPO:
    """
    Description
    --------------
    Class describing a DQN agent
    """
    
    def __init__(self, gamma=.9, categories=[4, 3, 3, 3, 2, 4], labels=[0, 1], min_queries=4):
        """
        Description
        --------------
        Constructor of class AgentDQN.
        
        Parameters & Attributes
        --------------
        gamma           : Float in ]0, 1[, the discount factor (default=0.9).
        categories      : List of length d where categories[i] is the number of categories feature i can take.
        labels          : List of the possible labels.
        min_queries     : Int, the minimum number of queries the agent has to perform before being allowed to report a label.
        d               : Int, the number of feature variables.
        b               : Int, the number of class labels.
        actions         : List of all actions.
        actions_queries : List of query actions.
        actions_report  : List of report actions.

        Returns
        --------------
        """
        
        self.gamma = gamma
        self.categories = categories
        self.labels = labels
        self.min_queries = min_queries
        self.d = len(categories)
        self.b = len(labels)
        self.actions = range(self.d + len(labels))
        self.actions_queries = range(self.d)
        self.actions_report = [self.d + label for label in labels]
        self.actor_critic = ActorCritic(input_size=np.sum(categories)+self.d, out=self.d+self.b)
        
    def actions_probas(self, state):
        """
        Description
        --------------
        Calculate the probabilities of the allowed actions at a state.
        
        Parameters
        --------------
        state   : Object of class State.
        
        Returns
        --------------
        actions_probas  : torch.tensor of size #Allowed_actions, with the allowed actions probabilities.
        actions_allowed : List of the allowed actions.
        value           : 1D torch.tensor, the state value estimated by the value head of the Actor-Critic network.
        """
        
        value, actions_output = self.actor_critic(torch.from_numpy(state.values_encoded))
        if len(state.observed) < self.min_queries:
            actions_allowed = list(state.unobserved)
            
        else:
            actions_allowed = list(state.unobserved) + [self.d+i for i in range(self.b)]
            
        actions_probas = F.softmax(actions_output[0, actions_allowed], dim=0)
        return actions_probas, actions_allowed, value
        
    def action(self, state):
        """
        Description
        --------------
        Choose an action at a state by sampling from the current stochastic policy.
        
        Parameters
        --------------
        state  : Object of class State.
        
        Returns
        --------------
        action           : Int in {0, ..., d-1, d, d+1}, action sampled from the stochastic policy of the Actor.
        action_log_proba : Float, the log probability correspoding to the performed action.
        actions_probas   : List of length #Allowed_actions with the estimated probabilities of the allowed actions.
        value            : 1D torch.tensor, the state value estimated by the value head of the Actor-Critic network.
        """
        
        actions_probas, actions_allowed, value = self.actions_probas(state)
        m = Categorical(actions_probas)
        index = m.sample()
        action, action_log_prob = actions_allowed[index], m.log_prob(index)
        return action, action_log_prob, actions_probas, value
    
    def action_greedy(self, state):
        """
        Description
        --------------
        Choose the action maximizing the stochastic policy at the state.
        
        Parameters
        --------------
        state  : Object of class State.
        
        Returns
        --------------
        action : Int in {0, ..., d-1, d, d+1}
        """
        
        with torch.no_grad():
            actions_probas, actions_allowed, _ = self.actions_probas(state)
            return actions_allowed[torch.argmax(actions_probas).item()]
        
        
    def train(self, env, n_train=1000, lr=3e-4, log_dir='runs_actor_critic/', n_save=1000, path_save='actor_critic_weights/', max_step=8, lambd=1e-3, 
              clip_grad=.1, clip_epsilon=.2):
        """
        Description
        --------------
        Explore the environment and train the agent.
        
        Parameters
        --------------
        env       : Object of class Environment.
        n_train   : Int, number of training episodes.
        lr        : Float, the learning rate.
        log_dir   : String, path of the folder where tensorboard events are stored.
        n_save    : Int, the number of episodes between two consecutive saved models.
        path_save : String, path of the directory where the models weights will be stored.
        max_step  : Int, the number of steps in an episode between two consecutive parameters update.
        lambd     : Float, the entropy loss parameter.
        clip_grad : Float in [0, 1] clipping the norm of the gradient to avoid over-optimistic updates.
        
        Returns
        --------------
        """
        
        writer = SummaryWriter(log_dir=log_dir)
        if not os.path.isdir(path_save):
            os.mkdir(path_save)
        
        optimizer = optim.Adam(self.actor_critic.parameters(), lr=lr)
        it = 0
        for episode in range(n_train):
            env.reset()
            state = deepcopy(env.state)
            episode_len = 0
            episode_rewards = 0
            while not env.done:
                rewards, values, log_probs = [], [], []
                step = 0
                entropy = 0
                while (not env.done) and (step <= max_step):
                    action, action_log_prob, actions_probas, value = self.action(state)
                    reward, next_state, _ = env.step(action)
                    episode_rewards += reward
                    rewards.append(reward)
                    values.append(value)
                    log_probs.append(action_log_prob.reshape(1, 1))
                    state = deepcopy(next_state)
                    entropy += -(actions_probas*torch.log(actions_probas)).sum()
                    step += 1
                    it += 1
                    episode_len += 1

                R = torch.tensor(0, dtype=torch.float32) if env.done else self.actor_critic(torch.from_numpy(state.values_encoded))[0].detach()
                size = len(values)
                values_target = [0 for i in range(size)]
                for t in range(size-1, -1, -1):
                    R = torch.tensor(rewards[t], dtype=torch.float32) + self.gamma*R
                    values_target[t] = R.reshape(1, 1)

                values_target = torch.cat(values_target)
                values_current = torch.cat(values)
                log_probs = torch.cat(log_probs)
                probs_chosen = torch.exp(log_probs)

                advantage = values_target - values_current
                critic_loss = 0.5*(advantage**2).mean()
#                 actor_loss = -(log_probs*advantage.detach()).mean()
                ratio = probs_chosen/probs_chosen.detach()
                advantage_detach = advantage.detach()
                surr1 = ratio*advantage_detach
                surr2 = torch.clamp(ratio, min = 1.-clip_epsilon, max = 1.+clip_epsilon)*advantage_detach
#                 actor_loss = -((probs_chosen/probs_chosen.detach())*advantage.detach()).mean()
                actor_loss = -torch.min(surr1, surr2).mean()
                entropy_loss = -entropy/size
                loss = actor_loss + critic_loss + lambd*entropy_loss
                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(self.actor_critic.parameters(), clip_grad)
                optimizer.step()

                writer.add_scalar('Losses/Actor', actor_loss.item(), it)
                writer.add_scalar('Losses/Critic', critic_loss.item(), it)
                writer.add_scalar('Losses/Entropy', entropy_loss, it)
                writer.add_scalar('Losses/Loss', loss.item(), it)
                
            writer.add_scalar('Episode/Return', episode_rewards, episode)
            writer.add_scalar('Episode/Length', episode_len, episode)
            if episode%n_save == 0:
                print('Episode : %d' %(episode))
                self.save_weights(path_save + 'actor_critic_weights_' + str(episode) + '.pth')
                        
        writer.close()
        
    def predict(self, env, data_point):
        """
        Description
        --------------
        Predict the label of a data point.
        
        Parameters
        --------------
        
        Returns
        --------------
        """
        
        env.reset(data_point)
        state = env.state
        while not env.done:
            action = self.action_greedy(state)
            env.step(action)
            state = env.state
        
        return action%self.d
        
    def test(self, env, n_test=1000):
        """
        Description
        --------------
        Test the agent on n_test data points generated by env.
        
        Parameters
        --------------
        env      : Object of class Environment.
        n_test   : Int, number of data points to test the agent on.
        
        Returns
        --------------
        accuracy : FLoat in [0, 1], the accuracy of the agent on this test.
        """
        
        valids = 0
        for i in range(n_test):
            data_point = env.generate()
            env.reset(data_point)
            label_pred, label_true = self.predict(env, data_point), env.label
            valids += (label_pred==label_true)
            
        return valids/n_test
    
    def save_weights(self, path):
        """
        Description
        --------------
        Save the agents q-network weights.
        
        Parameters
        --------------
        path: String, path to a .pth file containing the weights of a q-network.
        
        Returns
        --------------
        """
        
        torch.save(self.actor_critic.state_dict(), path)
    
    def load_weights(self, path):
        """
        Description
        --------------
        Load the weights of a q-network.
        
        Parameters
        --------------
        path: String, path to a .pth file containing the weights of a q-network.
        
        Returns
        --------------
        """
        
        self.actor_critic.load_state_dict(torch.load(path))
        
    def children(self, state):
        """
        Description
        --------------
        Give the possible outcomes of taking the greedy policy at the considered state.
        
        Parameters
        --------------
        state : Object of class State.
        
        Returns
        --------------
        children : Set of objects of class State.
        action   : Int, action taken at state with the agent policy.
        """
        
        children = []
        action = self.action_greedy(state)
        if action >= self.d: return children, action
        for category in range(self.categories[action]):
            values = state.values.copy()
            values[action] = category
            children.append(StateDQN(values, state.encoder, self.categories))

        return children, action
    
    def build_string_state(self, state):
        """
        Description
        --------------
        Build string representation of the agent decision (with parentheses) starting from state.
        
        Parameters
        --------------
        state : Object of class State.
        
        Returns
        --------------
        string : String representation of a tree.
        """
        
        l, action = self.children(state)
        if action >= self.d: return str(self.action_greedy(state)%self.d) + ''
        string = ''
        for child in l:
            string += '(X_' + str(action) + '=' + str(child.values[action]) + ' ' + self.build_string_state(child) + ') '

        return string
    
    def build_string(self, encoder):
        """
        Description
        --------------
        Build string representation of the agent decision.
        
        Parameters
        --------------
        
        Returns
        --------------
        string : String representation of a tree.
        """
        
        return '( ' + self.build_string_state(StateDQN([np.NaN for i in range(self.d)], encoder, self.categories)) + ')'
    
    def plot_tree(self, encoder):
        """
        Description
        --------------
        Plot the agent's decision tree.
        
        Parameters
        --------------
        
        Returns
        --------------
        nltk tree object, helpful to visualize the agent's decision tree policy.
        """

        return Tree.fromstring(self.build_string(encoder))


In [7]:
categories = [3, 5, 3, 3, 3, 5, 3, 5, 3, 3, 4, 3, 3, 3, 5]
labels = [0, 1]
d, b = len(categories), len(labels)
actions = range(d + len(labels))

# Feature variables are independent and uniform
generator = dict([(i, np.full(categories[i], 1/categories[i])) for i in range(len(categories))])

# Each query action costs -1
rewards_queries = dict([(i, -.5) for i in range(len(categories))])

# Define ethe encoder
encoder = Encoder(categories=categories)

In [8]:
env = EnvironmentDQN(generator, rewards_queries, encoder, r_plus=5, r_minus=-10, split=4)
agent = AgentPPO(categories=categories, min_queries=1)

In [9]:
agent.train(env, n_train=100000, lr=3e-4, lambd=1, max_step=30, log_dir='runs_ppo_min_queries_1/', path_save='ppo_weights_min_queries_1/')

Episode : 0
Episode : 1000
Episode : 2000
Episode : 3000
Episode : 4000
Episode : 5000
Episode : 6000
Episode : 7000
Episode : 8000
Episode : 9000
Episode : 10000
Episode : 11000
Episode : 12000
Episode : 13000
Episode : 14000
Episode : 15000
Episode : 16000
Episode : 17000
Episode : 18000
Episode : 19000
Episode : 20000
Episode : 21000
Episode : 22000
Episode : 23000
Episode : 24000
Episode : 25000
Episode : 26000
Episode : 27000
Episode : 28000
Episode : 29000
Episode : 30000
Episode : 31000
Episode : 32000
Episode : 33000
Episode : 34000
Episode : 35000
Episode : 36000
Episode : 37000
Episode : 38000
Episode : 39000
Episode : 40000
Episode : 41000
Episode : 42000
Episode : 43000
Episode : 44000
Episode : 45000
Episode : 46000
Episode : 47000
Episode : 48000
Episode : 49000
Episode : 50000
Episode : 51000
Episode : 52000
Episode : 53000
Episode : 54000
Episode : 55000
Episode : 56000
Episode : 57000
Episode : 58000
Episode : 59000
Episode : 60000
Episode : 61000
Episode : 62000
Episo